# Cloud Machine Learning Engine

In [6]:
import os
# use here the name of your own bucket
BUCKET = 'telemar-formazione-master-day6'

os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = 'europe-west1'

## Authorize CMLE
Cloud Machine Learning Engine needs to have access to train and test csv files.

In [2]:
%%bash
PROJECT_ID=$(gcloud config get-value core/project)

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-723822676475@cloud-ml.google.com.iam.gserviceaccount.com to access files in telemar-formazione-master-day6


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   237    0   237    0     0    190      0 --:--:--  0:00:01 --:--:--   190
Updated default ACL on gs://telemar-formazione-master-day6/
Updated ACL on gs://telemar-formazione-master-day6/flights/chapter8/output/testFlights-00001-of-00007.csv
Updated ACL on gs://telemar-formazione-master-day6/flights/chapter8/keys/origin.txt
Updated ACL on gs://telemar-formazione-master-day6/flights/chapter8/output/testFlights-00000-of-00007.csv
Updated ACL on gs://telemar-formazione-master-day6/flights/airports/airports.csv.gz
Updated ACL on gs://telemar-formazione-master-day6/flights/chapter8/keys/dest.txt
Updated ACL on gs://telemar-formazione-master-day6/flights/chapter8/output/testFlights-00004-of-00007.csv
Updated ACL on gs://telemar-formazione-master-day6/flights/chapter8/output/testFlights-00002-of-00007.csv
Updated ACL on gs://telemar-f

## Run a full training session on datalab
Code below launches a training job on Google Cloud CMLE. 
Note that:
- python package is contained in sub-folder flights: you can browse source cod using datalab;
- JOBNAME environmental variable is created starting from current date and time: check and find relevant lines of code;
- OUTPUT_DIR points to a specific folder in your bucket: it will contain training informations. 


In [3]:
%%bash

OUTPUT_DIR=gs://${BUCKET}/flights/chapter9/output
DATA_DIR=gs://${BUCKET}/flights/chapter8/output
ORIGIN_FILE=gs://${BUCKET}/flights/chapter8/keys/origin.txt
DEST_FILE=gs://${BUCKET}/flights/chapter8/keys/dest.txt
JOBNAME=flights_$(date -u +%y%m%d_%H%M%S)

PATTERN="Flights*"

echo "Launching training job ... trained model will be in $OUTPUT_DIR"

gsutil -m rm -rf $OUTPUT_DIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/flights/trainer \
  --job-dir=$OUTPUT_DIR \
  --staging-bucket=gs://$BUCKET \
  --runtime-version="1.6" \
  --scale-tier=STANDARD_1 \
  -- \
   --output_dir=$OUTPUT_DIR \
   --traindata $DATA_DIR/train$PATTERN \
   --evaldata $DATA_DIR/test$PATTERN   \
   --origin_file $ORIGIN_FILE          \
   --dest_file $DEST_FILE              \
   --num_training_epochs=5

Launching training job ... trained model will be in gs://telemar-formazione-master-day6/flights/chapter9/output
jobId: flights_180717_110157
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [flights_180717_110157] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe flights_180717_110157

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs flights_180717_110157


## Control CMLE log status
Browse to [https://console.cloud.google.com/mlengine](https://console.cloud.google.com/mlengine) and select your job

## Launch TensorBoard: visualize graph and metrics
We can keep track of the behavior of accuracy (on test set) and ()

In [7]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://'+BUCKET+'/flights/chapter9/output')
TensorBoard().list()

TensorBoard was started successfully with pid 26694. Click here to access it.

,logdir,pid,port
0,gs://telemar-formazione-master-day6/flights/ch...,23527,33609
1,gs://telemar-formazione-master-day6/flights/ch...,26650,44063
2,gs://telemar-formazione-master-day6/flights/ch...,26694,42597


In [4]:
# to stop TensorBoard
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 427
